In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Load the csv exported in Part I to a DataF
city_data= pd.read_csv("output_data/cities.csv")
city_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Punta Arenas,20,CL,1630304372,86,-53.1500,-70.9167,32.11,6.91
1,Ushuaia,20,AR,1630304384,75,-54.8000,-68.3000,35.26,12.66
2,Thompson,1,CA,1630304133,81,55.7435,-97.8558,48.36,0.00
3,Takoradze,100,GH,1630304385,89,4.8845,-1.7554,75.87,9.66
4,Jamestown,90,US,1630304384,98,42.0970,-79.2353,70.18,4.00


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]
figure = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
figure.add_layer(heat_layer)

figure

Figure(layout=FigureLayout(height='420px'))

In [5]:
narrowed_city = city_data.loc[(city_data["Max Temp"] < 80) & (city_data["Max Temp"] > 70) \
                                    & (city_data["Wind Speed"] < 10) \
                                    & (city_data["Cloudiness"] == 0)].dropna()
narrowed_city

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
67,Algiers,0,DZ,1630304406,78,36.7525,3.0420,71.42,2.30
192,Beyneu,0,KZ,1630304444,34,45.3167,55.2000,79.23,7.49
226,Göksun,0,TR,1630304454,29,38.0210,36.4973,70.75,1.50
347,Saint-Pierre,0,RE,1630304490,71,-21.3393,55.4781,76.68,4.61
381,Belyy Yar,0,RU,1630304500,44,53.6039,91.3903,76.89,4.47
468,Tambopata,0,PE,1630304525,67,-12.7333,-69.1833,71.17,2.82


In [6]:
hotels = narrowed_city[["City", "Country", "Lat", "Lng"]].copy()
hotels["Hotel Name"] = ""
hotels

,City,Country,Lat,Lng,Hotel Name
67,Algiers,DZ,36.7525,3.0420,
192,Beyneu,KZ,45.3167,55.2000,
226,Göksun,TR,38.0210,36.4973,
347,Saint-Pierre,RE,-21.3393,55.4781,
381,Belyy Yar,RU,53.6039,91.3903,
468,Tambopata,PE,-12.7333,-69.1833,


In [10]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}
 
for index, row in hotels.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try:
        hotels.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotels

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
67,Algiers,DZ,36.7525,3.0420,Sofitel Algiers Hamma Garden
192,Beyneu,KZ,45.3167,55.2000,NUR
226,Göksun,TR,38.0210,36.4973,Tuğçe Otel
347,Saint-Pierre,RE,-21.3393,55.4781,Lindsey Hôtel
381,Belyy Yar,RU,53.6039,91.3903,Pasportnyy Stol
468,Tambopata,PE,-12.7333,-69.1833,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
locations = hotels[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))